In [1]:
# Import libraries
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from torch import Tensor
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the SVHN Dataset as Digit Classes
training_set = loadmat('/content/drive/My Drive/Digits_Detector_in_PyTorch/data/train_32x32.mat')
testing_set = loadmat('/content/drive/My Drive/Digits_Detector_in_PyTorch/data/test_32x32.mat')

X_train = training_set['X']
X_test = testing_set['X']

X_train = np.swapaxes(X_train, 0, 3)
X_train = np.swapaxes(X_train, 1, 2)
X_train = np.swapaxes(X_train, 2, 3)

X_test = np.swapaxes(X_test, 0, 3)
X_test = np.swapaxes(X_test, 1, 2)
X_test = np.swapaxes(X_test, 2, 3)

y_train_raw = training_set['y']
y_test_raw = testing_set['y']

y_train = np.zeros((y_train_raw.shape[0], 1), dtype=int)
y_test = np.zeros((y_test_raw.shape[0], 1), dtype=int)

for i in range(y_train_raw.shape[0]):
  for j in range(10):
    y_train[i] = y_train_raw[i][0]

for i in range(y_test_raw.shape[0]):
  for j in range(10):
    y_test[i] = y_test_raw[i][0]


# Load the CIFAR-100 Dataset as Non-digit Classes
nd_training_set = loadmat('/content/drive/My Drive/Digits_Detector_in_PyTorch/data/train.mat')
nd_testing_set = loadmat('/content/drive/My Drive/Digits_Detector_in_PyTorch/data/test.mat')

X_train_nd = nd_training_set['data']
X_train_nd = np.reshape(X_train_nd, (X_train_nd.shape[0], 3, 32, 32))

X_test_nd = nd_testing_set['data']
X_test_nd = np.reshape(X_test_nd, (X_test_nd.shape[0], 3, 32, 32))

y_train_nd = np.zeros([X_train_nd.shape[0], 1]) + 10
y_test_nd = np.zeros([X_test_nd.shape[0], 1]) + 10

# Merge the training and testing sets to complete it
X_train = np.concatenate((X_train, X_train_nd), axis=0, out=None)
y_train = np.concatenate((y_train, y_train_nd), axis=0, out=None)

X_test = np.concatenate((X_test, X_test_nd), axis=0, out=None)
y_test = np.concatenate((y_test, y_test_nd), axis=0, out=None)

In [4]:
# Reformat the dataset into PyTorch dataloader format
training_dataset = TensorDataset(Tensor(X_train), Tensor(y_train.squeeze()).long())
training_dataset_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, num_workers=8)

test_dataset = TensorDataset(Tensor(X_test), Tensor(y_test.squeeze()).long())
test_dataset_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=8)


In [5]:
# Build VGG_16 Architecture in PyTorch
class VGG_16(nn.Module):
    def __init__(self, n_classes):
        super(VGG_16, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(1*1*512, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.pool(x)
        x = x.view(-1, 1 * 1 * 512)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x

    def predict(self, x):
        # a function to predict the labels of a batch of inputs
        x = F.softmax(self.forward(x, training=False))
        return x

    def accuracy(self, x, y):
        # a function to calculate the accuracy of label prediction for a batch of inputs
        #   x: a batch of inputs
        #   y: the true labels associated with x
        prediction = self.predict(x)
        maxs, indices = torch.max(prediction, 1)
        acc = 100 * torch.sum(torch.eq(indices.float(), y.float()).float())/y.size()[0]
        return acc.cpu().data[0]

In [ ]:
# # Training routine
# def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()

#                 # forward
#                 # track history if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)
#             if phase == 'train':
#                 scheduler.step()

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model


In [6]:
# Create VGG_16 Handler & Load it into GPU
vgg16 = VGG_16(11)

if torch.cuda.is_available():
  print("Using CUDA GPU")
  vgg16.cuda()

# Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# Train
for epoch in range(25):  # loop over the dataset multiple times
    print("Epoch #: ", (epoch+1))
    num_minibatches = 0

    running_loss = 0.0
    for i, data in enumerate(training_dataset_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        num_minibatches += 1

        if torch.cuda.is_available():
          inputs = inputs.cuda()
          labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        # running_loss += loss.item()
        # if i % 64 == 63:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 64))
        #     running_loss = 0.0

    running_loss = loss.item() / (64 * num_minibatches)
    print("Loss: ", loss.item())
    running_loss = 0.0

print('Training Complete')


# Test the performance
class_correct, class_total = 0, 0
with torch.no_grad():
    for i, data in enumerate(test_dataset_loader, 0):
        images, labels = data
        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for j in range(len(c)):
          class_total += 1
          if c[j].item():
            class_correct += 1

print('Accuracy : ', (100 * class_correct / class_total))


# Save the weights
PATH = "/content/drive/My Drive/Digits_Detector_in_PyTorch/vgg_16_wgts.pth"
torch.save(vgg16.state_dict(), PATH)


Using CUDA GPU
Epoch #:  1
Loss:  1.4036394357681274
Epoch #:  2
Loss:  1.2205415964126587
Epoch #:  3
Loss:  1.125879168510437
Epoch #:  4
Loss:  0.8247360587120056
Epoch #:  5
Loss:  0.9932150840759277
Epoch #:  6
Loss:  0.7410998344421387
Epoch #:  7
Loss:  0.6479732394218445
Epoch #:  8
Loss:  0.8475025296211243
Epoch #:  9
Loss:  0.5957813858985901
Epoch #:  10
Loss:  0.15343037247657776
Epoch #:  11
Loss:  0.05566499009728432
Epoch #:  12
Loss:  0.08436271548271179
Epoch #:  13
Loss:  0.1592126041650772
Epoch #:  14
Loss:  0.01789925992488861
Epoch #:  15
Loss:  0.03979805111885071
Epoch #:  16
Loss:  0.006007629446685314
Epoch #:  17
Loss:  0.12127238512039185
Epoch #:  18
Loss:  0.044808924198150635
Epoch #:  19
Loss:  0.03383266553282738
Epoch #:  20
Loss:  0.04532599821686745
Epoch #:  21
Loss:  0.012189471162855625
Epoch #:  22
Loss:  0.2453881949186325
Epoch #:  23
Loss:  0.06708887219429016
Epoch #:  24
Loss:  0.03387255221605301
Epoch #:  25
Loss:  0.020916646346449852
Tr